<a href="https://colab.research.google.com/github/jubayer27/AI_Text_Summarizer/blob/main/selfPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#libraries required

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [18]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [19]:
#distributing data in batch size

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle= True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle= True)

for X,y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [20]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [21]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[2].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Epoch 1
-------------------------------
loss: 2.300120  [   64/60000]
loss: 2.290934  [ 6464/60000]
loss: 2.276057  [12864/60000]
loss: 2.262367  [19264/60000]
loss: 2.243876  [25664/60000]
loss: 2.226271  [32064/60000]
loss: 2.213785  [38464/60000]
loss: 2.178912  [44864/60000]
loss: 2.163868  [51264/60000]
loss: 2.157033  [57664/60000]
Test Error: 
 Accuracy: 48.1%, Avg loss: 2.141642 

Epoch 2
-------------------------------
loss: 2.125481  [   64/60000]
loss: 2.116116  [ 6464/60000]
loss: 2.087025  [12864/60000]
loss: 2.085854  [19264/60000]
loss: 2.023601  [25664/60000]
loss: 1.992999  [32064/60000]
loss: 1.974818  [38464/60000]
loss: 1.970729  [44864/60000]
loss: 1.960172  [51264/60000]
loss: 1.904404  [57664/60000]
Test Error: 
 Accuracy: 55.4%, Avg loss: 1.863318 

Epoch 3
-------------------------------
loss: 1.847676  [   64/60000]
loss: 1.828141  [ 6464/60000]
loss: 1.709406  [12864/60000]
loss: 1.747290  [19264/60000]
loss: 1.658974  [25664/60000]
loss: 1.655548  [32064/600

IndexError: index 2 is out of bounds for dimension 0 with size 1